# Weak references

https://docs.python.org/3/library/weakref.html

We work with "raw" weak referneces first, in part for conceptual clarity. But most use of `weakref` facilities in Python should not be through weak reference objects (`weakref.ref` and `weakref.proxy`).

Instead, one usually uses the higher level facilities: `weakref.finalize`, `weakref.WeakSet`, `weakref.WeakKeyDictionary`, and `weakref.WeakValueDictionary`.

Sometimes, when building linked data structures (that store data in nodes), one uses "raw" weak references directly to avoid unnecessary strong cycles. That may be the most "basic" direct use of weak references in Python where it is totally reasonable to use them, and not reasonable to replace most uses with higher-level facilities.

You should reach for the higher level facilities first, in most cases.

In [1]:
import gc
import weakref

## Weak reference basics

Instances of unslotted classes are weak-referenceable:

In [2]:
class C: pass

In [3]:
c = C()

In [4]:
repr(c)

'<__main__.C object at 0x0000026531DE3310>'

In [5]:
print(c)

In [6]:
r = weakref.ref(c)

A reference's *referent* is the object it refers to. The `repr` of a `weakref.ref` shows whether its referent is alive or dead. Of course, this may change at any time:

In [7]:
r

<weakref at 0x0000026531E2C180; to 'C' at 0x0000026531DE3310>

Call a weak reference to access its referent. If the referent is dead, `None` is returned instead:

In [8]:
print(r())

In [9]:
r() is c

True

In [10]:
del c

In [124]:
print(r())

None


The `None` object itself is not weak-referenceable. One consequence of this is that there is no ambiguity (no semipredicate problem) in the meaning of the object returned by calling a `weakref.ref`:

In [12]:
n = None

In [13]:
n

In [14]:
rn = weakref.ref(n)

TypeError: cannot create weak reference to 'NoneType' object

`int`s are likewise not weak-referenceable:

In [15]:
rone = weakref.ref(1)

TypeError: cannot create weak reference to 'int' object

## Slotted classes and weak references

Instances of slotted classes are by default (and including in the case of empty slots) not weak-referenceable:

In [16]:
class CSlots: 
    __slots__ = ()

In [17]:
weakref.ref(CSlots())

TypeError: cannot create weak reference to 'CSlots' object

As with instance dictionaries, inheriting from a class whose instances are weak-referenceable gives a class whose instances are weak-referenceable regardless of whether the derived cass defines `__slots__` or what it lists in them.

However, the ability to be weakly referenced should not be confused with having an instance dictionary. On rare occasion, one may want a slotted class whose instances have instance dictionaries. Instances of such a class are not automatically weak-referenceable:

In [18]:
class DSlots: 
    __slots__ = ('__dict__',)

In [19]:
weakref.ref(DSlots())

TypeError: cannot create weak reference to 'DSlots' object

In [20]:
DSlots().__dict__

{}

## The `__weakref__` slot

To ensure instances of a class are weak referenceable, instances must have a `__weakref__` slot. This exists automatically if the class is unslotted, as does `__dict__`. If a class is slotted and all its direct and indirect base classes are slotted, then `__weakref__` must be defined to ensure weak-referenceability (and in CPython, in the absence of it, instances are guaranteed *not* to be weakly referenceable):

In [21]:
class WRSlots: 
    __slots__ = ('__weakref__',)

In [22]:
wr = WRSlots()

In [23]:
weakref.ref(wr)

<weakref at 0x000002653395BBA0; to 'WRSlots' at 0x0000026531DE2110>

In [24]:
weakref.ref(WRSlots())

<weakref at 0x000002653395B380; dead>

Note that the rules for slot duplication are not relaxed for `__weakref__`. If a base class is slotted and names `__weakref__` in its `__slots__`, then a directly or indirectly derived class *must not* also do so. If it does, then like with any other slot dupliacted from base to derived class, this is undefined behavior.

## Callbacks

Weak references can be created with callbacks. Assuming the weak reference itself still exists when the referent dies, the callback is called. The weak reference itself, and *not* the referent, is passed to the callback. The referent is, ideally, already dead and collected by the time the callback runs (though we will see that is not always the case). Whether or not the referent still exists, the weak reference sees it as dead at the time the callback is called. Consequently, weak reference callbacks do not resurrect objects. This is a very nice thing.

In [25]:
c2 = C()

In [26]:
wr = weakref.ref(c2, lambda x: print(f"{x}'s referent is dead!"))

In [27]:
wr

<weakref at 0x000002653395AFC0; to 'C' at 0x0000026532ED74F0>

In [28]:
del c2

<weakref at 0x000002653395AFC0; dead>'s referent is dead!


## Unhandled exceptions in callbacks

An unhandled exception in a callback, or that occurs *in attempting to call the callback*, cannot be caught. It is reported on standard error, in the same way as an unhandled exception in a `__del__` method.

In [29]:
wr

<weakref at 0x000002653395AFC0; dead>

In [30]:
wr2 = weakref.ref(C(), lambda: print("The referent is dead!"))

Exception ignored in: <function <lambda> at 0x00000265339A5750>
Traceback (most recent call last):
  File "C:\Users\User\AppData\Local\Temp\ipykernel_6984\3704447829.py", line 1, in <module>
TypeError: <lambda>() takes 0 positional arguments but 1 was given


In [31]:
wr2

<weakref at 0x0000026533960860; dead>

## Cycles

Weak reference callbacks are called when an object dies, even if the object has to be collected by the cyclic garbage collector.

In [32]:
cycle = C()

In [33]:
cycle.x = cycle

In [34]:
wrcycle = weakref.ref(cycle, lambda x: print("Referent (cycle) has died!"))

In [35]:
del cycle

In [36]:
gc.collect()

Referent (cycle) has died!


646

## `weakref.ref` equality comparison

In [37]:
class Box: 
    
    def __init__(self, value): 
        self.value = value
    
    def __repr__(self): 
        return f'{type(self).__name__}({self.value!r})'
    
    def __eq__(self, other): 
        if not isinstance(other, type(self)): 
            return NotImplemented
        return self.value == other.value

In [38]:
x = Box(1)
y = Box(1)

In [39]:
x == y

True

In [40]:
print(x)

Box(1)


### Calling `weakref.ref` may give an existing weak reference instance

In [41]:
r1 = weakref.ref(x)
r2 = weakref.ref(x)

In [42]:
r1 == r2

True

In [43]:
r1 is r2  # Not guaranteed.

True

In [44]:
r3 = weakref.ref(x, lambda _: print("Dead box"))

In [45]:
r3 is r1  # Guaranteed.

False

### Equality with live referents

If the referents are both alive and equal to each other, then the weak references to them are equal.

In [46]:
r3 == r1

True

In [47]:
s = weakref.ref(y)

In [48]:
r1 == s  # Weak references compare equal when their referents compare equal. 

True

Note that, because `_Box` instances are mutable and non-hashable, `weakref.ref`s to them are likewise unhashable.

In [49]:
hash(s)

TypeError: unhashable type: 'Box'

When referents stop being equal because of a change in state, weak refernces to them likewise stop being equal:

In [50]:
z = Box(1)

In [51]:
t = weakref.ref(z)

In [52]:
r1 == s == t

True

In [53]:
z.value = 2

In [54]:
print(z)

Box(2)


In [55]:
s == t

False

### Equality with dead referents

If the referents are dead, then weak references to them are only equal if they are the same `weakref.ref` object. Even separate `weakref.ref` objects that refer to the *same* referent become unequal when the referent dies.

In [56]:
del x

Dead box


In [57]:
r1 == r2

True

In [127]:
r1 is r2  # Because they are the same weak reference... which was NOT guaranteed!

True

In [59]:
r1 == r3  # Different weak references to the same dead referent.

False

### Weak references to live objects can *become* equal

In [60]:
print(y)

Box(1)


In [61]:
print(z)

Box(2)


In [62]:
s == t

False

In [63]:
z.value = 1

In [64]:
s == t

True

### Weak references to dead objects can become equal to themselves.

This happens in the pathological case of NaN-like objects that are not equal to themselves.

In [65]:
import math

In [66]:
nbox = Box(math.nan)

In [67]:
nbox == nbox

False

In [68]:
nbref = weakref.ref(nbox)

In [69]:
nbref == nbref

False

In [70]:
del nbox

In [71]:
nbref == nbref

True

## `weakref.ref` hashing

In [72]:
class LockBox: 
    
    def __init__(self, value): 
        self._value = value
    
    def __repr__(self): 
        return f'{type(self).__name__}({self.value!r})'
    
    def __eq__(self, other): 
        if not isinstance(other, type(self)): 
            return NotImplemented
        return self.value == other.value

    def __hash__(self): 
        return hash(self.value)

    @property
    def value(self): 
        return self._value

In [73]:
lb = LockBox(1)

In [74]:
print(lb)

LockBox(1)


In [75]:
g = weakref.ref(lb)

In [76]:
hash(g)

1

In [77]:
del lb

In [78]:
g

<weakref at 0x0000026533937470; dead>

The `weakref.ref` object cached the hash code and continues to return it:

In [79]:
hash(g)

1

Of course, that doesn't work if the hash code was never computed through the weak reference during the object's lifetime:

In [80]:
lb2 = LockBox(2)

In [81]:
h = weakref.ref(lb2)

In [82]:
del lb2

In [83]:
hash(h)

TypeError: weak object has gone away

## Weak reference callbacks and `__del__` methods

In [84]:
class SithHolocron: 
    def __init__(self, value): 
        self.value = value
    
    def __repr__(self): 
        return f'{type(self).__name__}({self.value!r})'
    
    def __eq__(self, other): 
        if not isinstance(other, type(self)): 
            return NotImplemented
        return self.value == other.value
    
    def __del__(self): 
        print(f"I see you are attempting to destroy {self!r}!")

### Without cycles

Normally, the `__del__` method is called before the weak reference callback:

In [85]:
sh = SithHolocron('ancient sith secrets')

In [86]:
sithweak = weakref.ref(sh, lambda x: print(f"{x!r}: Holocron destroyed!"))

In [87]:
del sh

I see you are attempting to destroy SithHolocron('ancient sith secrets')!
<weakref at 0x0000026531DCE2A0; dead>: Holocron destroyed!


This makes sense, since, ideally, the `__del__` method is called just before the object is destroyed, while, ideally, the weak reference callback is called just after the object is destroyed.

### With cycles

The above behavior is not guaranteed, and in practice, varies by implementation in the presence of cycles.

In *CPython*, the behavior demonstrated here is guaranteed.

In [88]:
sh2 = SithHolocron('modern sith secrets')

In [89]:
sh2.backup = sh2

In [90]:
modernsithweak = weakref.ref(sh2, lambda x: print(f"{x!r}: Holocron destroyed!"))

In [91]:
del sh2

In [92]:
gc.collect()  # 'dead' means not reachable

<weakref at 0x0000026531D37F10; dead>: Holocron destroyed!
I see you are attempting to destroy SithHolocron('modern sith secrets')!


956

Unintuitively, the weak refernece callback was called before the `__del__` method! This is because CPython's cyclic garbage collector sees to it that weak references are invalidated, and their callbacks call, before it proceeds to run finalizer methods (`__del__` methods).

## Weak reference callbacks and `__del__` methods: Resurrection

In [93]:
ringsofpower = []

In [94]:
class RingOfPower: 
    def __init__(self, value): 
        self.value = value
    
    def __repr__(self): 
        return f'{type(self).__name__}({self.value!r})'
    
    def __eq__(self, other): 
        if not isinstance(other, type(self)): 
            return NotImplemented
        return self.value == other.value
    
    def __del__(self): 
        print(f"I see you are attempting to destroy {self!r}!")
        ringsofpower.append(self)        

### Without cycles

In [95]:
onering = RingOfPower('one ring to rule them all ...')

In [96]:
weakring = weakref.ref(onering, lambda x: print(f"{x!r} has casted the ring into the fire!"))

In [97]:
del onering

I see you are attempting to destroy RingOfPower('one ring to rule them all ...')!


In [98]:
weakring

<weakref at 0x000002653398A1B0; to 'RingOfPower' at 0x0000026532F17FA0>

In [99]:
print(ringsofpower)

[RingOfPower('one ring to rule them all ...')]


In [100]:
ringsofpower.clear()

<weakref at 0x000002653398A1B0; dead> has casted the ring into the fire!


### With cycles

In [101]:
wkring = RingOfPower("The Witch King's ring")

In [102]:
wkring.backup = wkring 

In [103]:
wkref = weakref.ref(wkring, lambda x: print(f"{x!r} (not a man) has killed the Witch King."))

In [104]:
del wkring

In [105]:
gc.collect()

<weakref at 0x0000026531E367A0; dead> (not a man) has killed the Witch King.
I see you are attempting to destroy RingOfPower("The Witch King's ring")!


0

In [106]:
print(ringsofpower)  # BUT WE KILLED HIM!!! (Don't resurrect objects)

[RingOfPower("The Witch King's ring")]


## Lifetime of weak references

If you delete the weak reference before you delete the referent, the callback will not be called:

In [107]:
nlb = LockBox('The budget surplus')

In [108]:
lbwref = weakref.ref(nlb, lambda _: print("There goes the lockbox"))

In [109]:
del nlb

There goes the lockbox


In [110]:
nlb2 = LockBox('The budget deficit')

In [111]:
lb2wref = weakref.ref(nlb2, lambda _: print("There goes the lockbox!"))

In [112]:
del lb2wref

In [113]:
del nlb2

This motivates weakref finalizers.

## Weak reference finalizers

In [114]:
bigbox = Box('BIG!')

In [115]:
bigboxfin = weakref.finalize(bigbox, lambda: print("Destroyed a big box."))

In [116]:
del bigboxfin

In [117]:
del bigbox

Destroyed a big box.


In [118]:
lockbox = LockBox('Hidden costs')

In [119]:
lockboxfin = weakref.finalize(lockbox, lambda x: print(f'There were... {x}.'), lockbox.value)

In [120]:
del lockbox

There were... Hidden costs.


If a weakref finalizer holds a strong reference to the object it is supposed to "finalize", then the object can never be collected. This leaks the object (and the finalizer):

In [121]:
lockbox2 = LockBox('Big hidden costs.')

In [122]:
lockbox2fin = weakref.finalize(lockbox2, lambda x: print(f'There were... {x.value}.'), lockbox2)

In [123]:
del lockbox2, lockbox2fin